We can provide both relaxed and strict F1 score.



strict F1 score can be calculated directly using seqeval.

In [ ]:
!pip install seqeval

In [ ]:
#!pip install seqeval
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

# Example ground truth (true labels) and predictions
true_labels = [['O', 'I-PER', 'I-PER', 'O'], ['I-LOC'], ['I-LOC']]
predicted_labels = [['O', 'I-PER', 'I-PER', 'O'], ['I-LOC'], ['O']]

#predictions = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
#references = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

# Generate a classification report
report = classification_report(true_labels, predicted_labels)
f1_score = f1_score(true_labels, predicted_labels)

print("F1 Score:", f1_score)

#results = seqeval.compute(predictions=predictions, references=references)
# Print the F1-score
print("Classification Report:\n", report)

consier token-level

In [ ]:
def calculate_token_level_f1(true_labels, predicted_labels):
    total_tokens = len(true_labels)
    tp, fp, fn = 0, 0, 0

    for true_label, pred_label in zip(true_labels, predicted_labels):
        for true_token, pred_token in zip(true_label, pred_label):
            if true_token == pred_token and true_token != 'O':
                tp += 1
            elif true_token == 'O' and pred_token != 'O':
                fp += 1
            elif true_token != 'O' and pred_token == 'O':
                fn += 1

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1_score

# Example usage:
#true_labels = [['B-PER', 'I-PER', 'O', 'B-LOC', 'O']]
#predicted_labels = [['B-PER', 'O', 'O', 'B-LOC', 'O']]

#true_labels = [['O', 'B-PER', 'I-PER', 'O', 'B-LOC', 'I-LOC']]
#predicted_labels = [['O', 'I-PER', 'I-PER', 'O', 'B-LOC', 'O']]

token_level_f1 = calculate_token_level_f1(gold_labels, pred_labels)
print("Token-level F1 Score:", token_level_f1)


For relaxed F1 score:

International Workshop on Semantic Evaluation (SemEval)

The SemEval’13 introduced four different ways to measure precision/recall/f1-score results based on the metrics defined by MUC (Message Understanding Conference).

    Strict: exact boundary surface string match and entity type;

    Exact: exact boundary match over the surface string, regardless of the type;

    Partial: partial boundary match over the surface string, regardless of the type;

    Type: some overlap between the system tagged entity and the gold annotation is required;

each of these ways to measure the performance accounts for correct, incorrect, partial, missed and spurious in different ways.

Note you can find the complete code for this evaluation metrics on this repository:

    https://github.com/davidsbatista/NER-Evaluation

You can find a more detailed explanation in the following blog post:

    http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/



In [ ]:
!git clone https://github.com/davidsbatista/NER-Evaluation

change folder name "NER-Evaluation" to "NER_Evaluation"

In [ ]:
from copy import deepcopy
from collections import defaultdict

import NER_Evaluation.ner_evaluation.ner_eval

from NER_Evaluation.ner_evaluation.ner_eval import collect_named_entities
from NER_Evaluation.ner_evaluation.ner_eval import compute_metrics
from NER_Evaluation.ner_evaluation.ner_eval import compute_precision_recall_wrapper

In [ ]:
file_path_new = '/content/impacts_golden_labels.txt'

gold_labels = []  # Initialize an empty list for the predicted labels

with open(file_path_new, 'r', encoding='utf-8') as file:  # Ensure proper encoding
    for line in file:
        line_labels = line.strip().split()  # Split each line into labels
        gold_labels.append(line_labels)  # Append the list of labels to the main list

# Show the first 5 entries of pred_labels to verify the operation
gold_labels[:5]

In [ ]:
# Re-attempt processing the file to create a nested list structure for pred_labels
file_path_new = '/content/predictions-GPT4-impacts-dpr.txt'

pred_labels = []  # Initialize an empty list for the predicted labels

with open(file_path_new, 'r', encoding='utf-8') as file:  # Ensure proper encoding
    for line in file:
        line_labels = line.strip().split()  # Split each line into labels
        pred_labels.append(line_labels)  # Append the list of labels to the main list

# Show the first 5 entries of pred_labels to verify the operation
#pred_labels[:5]


In [ ]:
len(gold_labels)

In [ ]:
len(pred_labels)

In [ ]:
metrics_results = {'correct': 0, 'incorrect': 0, 'partial': 0,
                   'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0}

# overall results
results = {'strict': deepcopy(metrics_results),
           'ent_type': deepcopy(metrics_results),
           'partial':deepcopy(metrics_results),
           'exact':deepcopy(metrics_results)
          }


# results aggregated by entity type
evaluation_agg_entities_type = {e: deepcopy(results) for e in ['Social_Impacts', 'Clinical_Impacts']}

for true_ents, pred_ents in zip(gold_labels, pred_labels):

    # compute results for one message
    tmp_results, tmp_agg_results = compute_metrics(
        collect_named_entities(true_ents), collect_named_entities(pred_ents),  ['Social_Impacts', 'Clinical_Impacts']
    )

    #print(tmp_results)

    # aggregate overall results
    for eval_schema in results.keys():
        for metric in metrics_results.keys():
            results[eval_schema][metric] += tmp_results[eval_schema][metric]

    # Calculate global precision and recall

    results = compute_precision_recall_wrapper(results)


    # aggregate results by entity type

    for e_type in ['Social_Impacts', 'Clinical_Impacts']:

        for eval_schema in tmp_agg_results[e_type]:

            for metric in tmp_agg_results[e_type][eval_schema]:

                evaluation_agg_entities_type[e_type][eval_schema][metric] += tmp_agg_results[e_type][eval_schema][metric]

        # Calculate precision recall at the individual entity level

        evaluation_agg_entities_type[e_type] = compute_precision_recall_wrapper(evaluation_agg_entities_type[e_type])



I usually use "ent_type" or "strict"

In [ ]:
precision = results['ent_type']['precision']
recall = results['ent_type']['recall']
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

In [ ]:
results['strict']['spurious']

In [ ]:
results['strict']

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1_score